<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/%EC%9E%84%EB%B2%A0%EB%94%A9_%EB%AA%A8%EB%8D%B8_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [4]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size=len(tokenizer.word_index)+1
print('단어 집합 : ', vocab_size)

단어 집합 :  16


In [5]:
X_encoded=tokenizer.texts_to_sequences(sentences)
print('정수 인코딩 결과 : ', X_encoded)

정수 인코딩 결과 :  [[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [6]:
max_len=max(len(word) for word in X_encoded)
print('단어 중 최대 길이 : ', max_len)

단어 중 최대 길이 :  4


In [7]:
X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train=np.array(y_train)
print('패딩 결과 : ', X_train)

패딩 결과 :  [[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten

embedding_dim=4
model=Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten()) #입력 데이터를 1차원으로 평탄화 -> 이후의 레이어에서 처리하기 쉽게 만들어준다.
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 6s - loss: 0.6963 - acc: 0.5714 - 6s/epoch - 6s/step
Epoch 2/100
1/1 - 0s - loss: 0.6950 - acc: 0.5714 - 8ms/epoch - 8ms/step
Epoch 3/100
1/1 - 0s - loss: 0.6938 - acc: 0.5714 - 8ms/epoch - 8ms/step
Epoch 4/100
1/1 - 0s - loss: 0.6925 - acc: 0.5714 - 8ms/epoch - 8ms/step
Epoch 5/100
1/1 - 0s - loss: 0.6912 - acc: 0.7143 - 8ms/epoch - 8ms/step
Epoch 6/100
1/1 - 0s - loss: 0.6900 - acc: 0.7143 - 8ms/epoch - 8ms/step
Epoch 7/100
1/1 - 0s - loss: 0.6887 - acc: 0.7143 - 8ms/epoch - 8ms/step
Epoch 8/100
1/1 - 0s - loss: 0.6874 - acc: 0.7143 - 10ms/epoch - 10ms/step
Epoch 9/100
1/1 - 0s - loss: 0.6861 - acc: 0.7143 - 12ms/epoch - 12ms/step
Epoch 10/100
1/1 - 0s - loss: 0.6849 - acc: 0.7143 - 8ms/epoch - 8ms/step
Epoch 11/100
1/1 - 0s - loss: 0.6836 - acc: 0.7143 - 8ms/epoch - 8ms/step
Epoch 12/100
1/1 - 0s - loss: 0.6823 - acc: 0.7143 - 8ms/epoch - 8ms/step
Epoch 13/100
1/1 - 0s - loss: 0.6810 - acc: 0.7143 - 7ms/epoch - 7ms/step
Epoch 14/100
1/1 - 0s - loss: 0.6798 - acc: 0

In [9]:
#훈련데이터가 적을 경우 미리 사전에 학습된 임베딩벡터(Word2Vec, GloVe)들을 사용한다.
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
zf=zipfile.ZipFile('glove.6B.zip')
zf.extractall()
zf.close()

In [10]:
embedding_dict=dict()
f=open('glove.6B.100d.txt', encoding='utf8')

for line in f: #파일에서 한 줄씩 읽어오기
  word_vector=line.split() #라인 별로 공백 기준으로 나눠서 word_vector에 넣어준다.
  word=word_vector[0] #word_vector의 첫 인덱스에는 단어가 적혀져 있다.

  #100개의 값을 가지는 array로 변환
  word_vector_arr=np.asarray(word_vector[1:],dtype='float32')
  embedding_dict[word]=word_vector_arr
f.close()

print('%s개의 Embedding vector가 있습니다. ' %len(embedding_dict))

400000개의 Embedding vector가 있습니다. 


In [11]:
print(embedding_dict['respectable'])
print('벡터의 차원 수 : ', len(embedding_dict['respectable']))

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [12]:
#벡터의 차원수 만큼을 열(column)로 가지는 행렬 생성
embedding_matrix=np.zeros((vocab_size, 100))
print('임베딩 행렬의 크기 : ', embedding_matrix.shape)

임베딩 행렬의 크기 :  (16, 100)


In [13]:
print(tokenizer.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [14]:
#사전 훈련된 great의 정수와 GloVe에서의 great의 벡터값을 확인한다
print('앞서 맵핑된 정수 : ', tokenizer.word_index['great'])
print('GloVe의 벡터값 : ', embedding_dict['great'])

앞서 맵핑된 정수 :  2
GloVe의 벡터값 :  [-0.013786   0.38216    0.53236    0.15261   -0.29694   -0.20558
 -0.41846   -0.58437   -0.77355   -0.87866   -0.37858   -0.18516
 -0.128     -0.20584   -0.22925   -0.42599    0.3725     0.26077
 -1.0702     0.62916   -0.091469   0.70348   -0.4973    -0.77691
  0.66045    0.09465   -0.44893    0.018917   0.33146   -0.35022
 -0.35789    0.030313   0.22253   -0.23236   -0.19719   -0.0053125
 -0.25848    0.58081   -0.10705   -0.17845   -0.16206    0.087086
  0.63029   -0.76649    0.51619    0.14073    1.019     -0.43136
  0.46138   -0.43585   -0.47568    0.19226    0.36065    0.78987
  0.088945  -2.7814    -0.15366    0.01015    1.1798     0.15168
 -0.050112   1.2626    -0.77527    0.36031    0.95761   -0.11385
  0.28035   -0.02591    0.31246   -0.15424    0.3778    -0.13599
  0.2946    -0.31579    0.42943    0.086969   0.019169  -0.27242
 -0.31696    0.37327    0.61997    0.13889    0.17188    0.30363
 -1.2776     0.044423  -0.52736   -0.88536   -0.19428   -0

In [15]:
#word_index의 모든 단어에 대해서 GloVe의 임베딩 벡터들을 맵핑한다.
for word, index in tokenizer.word_index.items():
  vector_value=embedding_dict[word]
  embedding_matrix[index]=vector_value

In [16]:
embedding_matrix[2]

array([-0.013786  ,  0.38216001,  0.53236002,  0.15261   , -0.29694   ,
       -0.20558   , -0.41846001, -0.58437002, -0.77354997, -0.87866002,
       -0.37858   , -0.18516   , -0.12800001, -0.20584001, -0.22925   ,
       -0.42598999,  0.3725    ,  0.26076999, -1.07019997,  0.62915999,
       -0.091469  ,  0.70348001, -0.4973    , -0.77691001,  0.66044998,
        0.09465   , -0.44893   ,  0.018917  ,  0.33146   , -0.35021999,
       -0.35789001,  0.030313  ,  0.22253001, -0.23236001, -0.19719   ,
       -0.0053125 , -0.25848001,  0.58081001, -0.10705   , -0.17845   ,
       -0.16205999,  0.087086  ,  0.63028997, -0.76648998,  0.51618999,
        0.14072999,  1.01900005, -0.43136001,  0.46138   , -0.43584999,
       -0.47567999,  0.19226   ,  0.36065   ,  0.78987002,  0.088945  ,
       -2.78139997, -0.15366   ,  0.01015   ,  1.17980003,  0.15167999,
       -0.050112  ,  1.26259995, -0.77526999,  0.36030999,  0.95761001,
       -0.11385   ,  0.28035   , -0.02591   ,  0.31246001, -0.15

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

output_dim=100 #GloVe의 임베딩 벡터들은 모두 100차원이다.
model=Sequential()
model.add(Embedding(vocab_size, output_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.8397 - acc: 0.4286 - 700ms/epoch - 700ms/step
Epoch 2/100
1/1 - 0s - loss: 0.8124 - acc: 0.4286 - 7ms/epoch - 7ms/step
Epoch 3/100
1/1 - 0s - loss: 0.7860 - acc: 0.4286 - 9ms/epoch - 9ms/step
Epoch 4/100
1/1 - 0s - loss: 0.7603 - acc: 0.5714 - 8ms/epoch - 8ms/step
Epoch 5/100
1/1 - 0s - loss: 0.7356 - acc: 0.7143 - 8ms/epoch - 8ms/step
Epoch 6/100
1/1 - 0s - loss: 0.7117 - acc: 0.7143 - 6ms/epoch - 6ms/step
Epoch 7/100
1/1 - 0s - loss: 0.6888 - acc: 0.7143 - 7ms/epoch - 7ms/step
Epoch 8/100
1/1 - 0s - loss: 0.6667 - acc: 0.8571 - 7ms/epoch - 7ms/step
Epoch 9/100
1/1 - 0s - loss: 0.6456 - acc: 0.8571 - 6ms/epoch - 6ms/step
Epoch 10/100
1/1 - 0s - loss: 0.6254 - acc: 0.8571 - 6ms/epoch - 6ms/step
Epoch 11/100
1/1 - 0s - loss: 0.6061 - acc: 0.8571 - 7ms/epoch - 7ms/step
Epoch 12/100
1/1 - 0s - loss: 0.5877 - acc: 0.8571 - 7ms/epoch - 7ms/step
Epoch 13/100
1/1 - 0s - loss: 0.5701 - acc: 0.8571 - 8ms/epoch - 8ms/step
Epoch 14/100
1/1 - 0s - loss: 0.5533 - acc:

In [18]:
#엘모(Embedding from Language Model) ELMo
#biLM(Bidirectional Language Model) -> 순방향과 역방향 RNN을 사용하는 두 가지가 있으며 문맥을 고려한 임베딩 방법(합성곱 신경망을 사용, embedding층이나 word2vec사용 x. 함께 사용은 concatenate해서 가능)

임베딩 벡터의 시각화

In [19]:
!python -m gensim.scripts.word2vec2tensor

usage: word2vec2tensor.py
       [-h]
       -i
       INPUT
       -o
       OUTPUT
       [-b]
word2vec2tensor.py: error: the following arguments are required: -i/--input, -o/--output


In [20]:
!python -m gensim.scripts.word2vec2tensor --input word2vec --output word2vec #미리 저장한 모델 이름
#https://projector.tensorflow.org/ 여기에 접속해서 load 버튼 누른 후 파일 2개를 넣는다.

2023-07-24 04:56:57,051 - word2vec2tensor - INFO - running /usr/local/lib/python3.10/dist-packages/gensim/scripts/word2vec2tensor.py --input word2vec --output word2vec
2023-07-24 04:56:57,051 - keyedvectors - INFO - loading projection weights from word2vec
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/gensim/scripts/word2vec2tensor.py", line 94, in <module>
    word2vec2tensor(args.input, args.output, args.binary)
  File "/usr/local/lib/python3.10/dist-packages/gensim/scripts/word2vec2tensor.py", line 68, in word2vec2tensor
    model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_model_path, binary=binary)
  File "/usr/local/lib/python3.10/dist-packages/gensim/models/keyedvectors.py", line 1719, in load_word2vec_format
    return _lo

문서 벡터를 이용한 추천시스템(단어 벡터로 변환하고 평균을 구하여 문서 벡터를 얻는다)

In [3]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from PIL import Image
from io import BytesIO
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/data.csv", filename="data.csv")
df=pd.read_csv('data.csv')
print('전체 문서의 수 : ', len(df))

전체 문서의 수 :  2382


In [5]:
df[:5]

,Unnamed: 0.1,Desc,Unnamed: 0,author,genre,image_link,rating,title
0,0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing
4,4,Amazing book. And I joined Steve Jobs and many...,4.0,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
def _removeNoneAscii(s):
  return "".join(i for i in s if ord(i)<128) #아스키코드 127까지 모두 반환한다.

def make_lower_case(text):
  return text.lower()

def remove_stop_words(text):
  text=text.split()
  stops=set(stopwords.words('english'))
  text=[w for w in text if not w in stops]
  text=" ".join(text) #띄어쓰기를 넣어주고 모두 한꺼번에 반환한다.
  return text

def remove_html(text):
  html_pattern=re.compile('<.*?>') #정규식 객체
  return html_pattern.sub(r'',text)

def remove_punctuation(text):
  tokenizer=RegexpTokenizer(r'[a-zA-Z]+')
  text=tokenizer.tokenize(text)
  text=" ".join(text)
  return text

df['cleaned'] = df['Desc'].apply(_removeNoneAscii)
df['cleaned'] = df.cleaned.apply(make_lower_case)
df['cleaned'] = df.cleaned.apply(remove_stop_words)
df['cleaned'] = df.cleaned.apply(remove_punctuation)
df['cleaned'] = df.cleaned.apply(remove_html)


In [8]:
df['cleaned'][:5]

0    know power shifting west east north south pres...
1    following success accidental billionaires mone...
2    tap power social software networks build busin...
3    william j bernstein american financial theoris...
4    amazing book joined steve jobs many akio morit...
Name: cleaned, dtype: object

In [9]:
df['cleaned'].replace('',np.nan, inplace=True) #빈 값을 nan으로 변환
df=df[df['cleaned'].notna()] #해당 행을 제거
print(len(df))

2381


In [10]:
corpus=[]
for words in df['cleaned']:
  corpus.append(words.split()) #단어들을 코퍼스에 넣는다.

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300') #사전에 훈련된 word2vec의 벡터들을 불러오기

[=================================================-] 99.8% 1659.0/1662.8MB downloaded

In [16]:
word2vec_model=Word2Vec(vector_size=300, window=5, min_count=2, workers=-1)
word2vec_model.build_vocab(corpus)
word2vec_model.train(corpus, total_examples=word2vec_model.corpus_count, epochs=15)

(0, 0)

In [17]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
vocab_size=20000

(X_train, y_train), (X_test, y_test)=imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 : ', len(X_train))
print('테스트용 리뷰 개수 : ', len(X_test))

훈련용 리뷰 개수 :  25000
테스트용 리뷰 개수 :  25000


In [21]:
print('훈련용 리뷰의 평균 길이 : {}'.format(np.mean(list(map(len,X_train)), dtype='int'))) #X_train의 요소들을 길이로 변환 후 리스트로 바꾸고 평균을 구한다.
print('테스트용 리뷰의 평균 길이 : {}'.format(np.mean(list(map(len, X_test)), dtype='int')))

훈련용 리뷰의 평균 길이 : 238
테스트용 리뷰의 평균 길이 : 230


In [22]:
max_len=400
X_train=pad_sequences(X_train, maxlen=max_len)
X_test=pad_sequences(X_test, maxlen=max_len)

In [23]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim=64

model=Sequential()
model.add(Embedding(vocab_size, embedding_dim))

model.add(GlobalAveragePooling1D()) #모든 단어 벡터의 평균을 구한다.
model.add(Dense(1, activation='sigmoid'))

es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) #과적합 방지를 위해 콜백함수 earlystopping을 사용하여 적절한 시점에 조기종료를 시킨다.
#monitor : 학습 조기종료를 위해 관찰하는 항목. val_loss와 val_accuracy가 주로 사용된다.
#mode : 관찰항목에 대해 개선이 없다고 판단하기 위한 기준. val_loss일 경우 오차 값이 감소되지 않을 경우 종료 ->min, val_accuracy는 정확도가 올라가야 하므로 ->max
#patience : 개선이 되지 않는다고 바로 끄지 않고 최소의 epoch를 설정
mc=ModelCheckpoint('embedding_average_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)\
#맨 앞 문자열은 모델을 저장할 경로 'asdfasdf.h5'로 저장형식.
#monitor : 모델을 저장할 때, 기준이 되는 값. 마찬가지로 val_loss, val_accuracy
#mode : earlystopping과 같은 원리
#verbose : 1이면 저장되었다고 표시됨. 0은 표시 없이 저장됨
#save_best_only : True -> monitor기준으로 가장 좋은 값이 저장, False -> 매 epoch마다 모델이 filepath{epoch}으로 저장됨
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=10, batch_size=32, callbacks=[es, mc], validation_split=0.2)
#validation_split은 학습 데이터셋이 80%, 테스트 데이터셋이 20%
#batch_size는 데이터를 32개씩 쪼개서 학습

Epoch 1/10
624/625 [============================>.] - ETA: 0s - loss: 0.6299 - acc: 0.7221
Epoch 1: val_acc improved from -inf to 0.81760, saving model to embedding_average_model.h5
625/625 [==============================] - 92s 139ms/step - loss: 0.6296 - acc: 0.7224 - val_loss: 0.5232 - val_acc: 0.8176
Epoch 2/10
615/625 [============================>.] - ETA: 0s - loss: 0.4330 - acc: 0.8541
Epoch 2: val_acc improved from 0.81760 to 0.86720, saving model to embedding_average_model.h5
625/625 [==============================] - 31s 50ms/step - loss: 0.4318 - acc: 0.8545 - val_loss: 0.3799 - val_acc: 0.8672
Epoch 3/10
624/625 [============================>.] - ETA: 0s - loss: 0.3200 - acc: 0.8910
Epoch 3: val_acc improved from 0.86720 to 0.87720, saving model to embedding_average_model.h5
625/625 [==============================] - 27s 43ms/step - loss: 0.3199 - acc: 0.8911 - val_loss: 0.3253 - val_acc: 0.8772
Epoch 4/10
623/625 [============================>.] - ETA: 0s - loss: 0.2628 -

In [26]:
loaded_model=load_model('embedding_average_model.h5')
print("\n 테스트 정확도 : " ,(loaded_model.evaluate(X_test, y_test))) #정확도 검증-> 오차율, 정확도

782/782 [==============================] - 3s 3ms/step - loss: 0.2918 - acc: 0.8877

 테스트 정확도 :  [0.2918189764022827, 0.8876799941062927]


In [28]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL" -O dart.csv && rm -rf /tmp/cookies.txt
#data.csv 파일 다운로드

--2023-07-24 05:49:58--  https://docs.google.com/uc?export=download&confirm=t&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL
Resolving docs.google.com (docs.google.com)... 172.253.117.102, 172.253.117.113, 172.253.117.139, ...
Connecting to docs.google.com (docs.google.com)|172.253.117.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/rufvebcf54vghf0n18psvgrnb55ie0v4/1690177725000/17609157229046208934/*/1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL?e=download&uuid=bdbed11a-fed2-4848-aed0-03314b9a8310 [following]
--2023-07-24 05:49:58--  https://doc-04-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/rufvebcf54vghf0n18psvgrnb55ie0v4/1690177725000/17609157229046208934/*/1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL?e=download&uuid=bdbed11a-fed2-4848-aed0-03314b9a8310
Resolving doc-04-7k-docs.googleusercontent.com (doc-04-7k-docs.googleusercontent.com)... 74.125.142.1

In [29]:
#형태소 분석기 Mecab설치
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 48.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 15.2 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1232 sha256=619459610bccb24f5195ef2a921aacdf61ad1ebf7bdec8c3ce69d537442e2840
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  1340k      0  0:00:01  0:00:01 --:--:-- 2283k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/ex

In [30]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [32]:
df=pd.read_csv('/content/dart.csv', sep=',')
df=df.replace('',np.nan)
df=df.dropna()
df

,code,market,name,business
0,000020,KOSPI,동화약품,II. 사업의 내용\n1. 사업의 개요\n가. 일반적인 사항\n기업회계기준서 제11...
1,000040,KOSPI,KR모터스,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n수출주력시장인 유럽 불...
2,000050,KOSPI,경방,II. 사업의 내용\n1. 사업의 개요\n(1) 산업의 특성\n[섬유사업부문]\n면...
3,000060,KOSPI,메리츠화재,"II. 사업의 내용\n1. 사업의 개요\n가. 산업의 특성, 성장성, 경기변동의 특..."
4,000070,KOSPI,삼양홀딩스,"II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n지주회사(持株會社, H..."
...,...,...,...,...
2582,950130,KOSDAQ,엑세스바이오,II. 사업의 내용\n1. 사업의 개\n요\n당사는 체외진단 기술을 토대로 말라리아...
2583,950140,KOSDAQ,잉글우드랩,II. 사업의 내용\n1. 사업의 개요\n가. 화장품 ODM산업 개요\nODM은 O...
2584,950160,KOSDAQ,코오롱티슈진,II. 사업의 내용\n당사는 1999년 6월 9일 세포 유전자 기술 Platform...
2585,950170,KOSDAQ,JTC,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n(1) 산업의 특성\n...


In [34]:
mecab=Mecab()
tagged_corpus_list=[]
for index, row in tqdm(df.iterrows(),total=len(df)): #bar형식으로 진행상황을 하고 iterrows()로 각 행의 정보들을 반환한다. total로 전체 진행 개수를 알 수 있다.
  text=row['business'] #row에 행의 정보들이 담겨있다.
  tag=row['name']
  tagged_corpus_list.append(TaggedDocument(tags=[tag],words=mecab.morphs(text)))
  #tags에는 tag에 담은 제목, words에는 형태소 분석기로 형태소 별로 나눈 text가 들어간다.
print('문서의 수 : ', len(tagged_corpus_list))

100%|██████████| 2295/2295 [01:37<00:00, 23.63it/s]

문서의 수 :  2295


In [38]:
#doc2vec 학습 및 테스트
from gensim.models import doc2vec

In [39]:
model=doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

model.build_vocab(tagged_corpus_list)

model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

model.save('dart.doc2vec')